# Molecular dynamics hands-on Python coding practices

This Jupyter Notebook file serves as a gateway to enter the realm of Python programming and molecular dynamics. We'll break down the basic MD loop to several code cells, step-by-step. The main purpose of this coding assignment is to horn your Python programming skills. The first part will be creating multiple arrays, as containers, to store atoms' information such as positions, velocity, acceleration, interatomic relative position vector, forces, etc. The second part will be run time iterations to calculate atomic foreces, accelerations, and update atomic positions. Note that the material parameters are arbitrarily set up. You'll need to use realistic parameters if you want to simulate a real system.

by Hui-Chia Yu, 2025

---

### Part 1: Modify the single phase code and a two phase code

In this practice, we distinguish atoms to A-type and B-type. B-type has a mass twice that of A. Now we have three types of interatomic bonding:
$\varepsilon_{AA}$, $\varepsilon_{AB}$, and $\varepsilon_{BB}$.
In the first test run, we have 
$\varepsilon_{AA} = 0.25$, $\varepsilon_{AB}= 2.5$, and $\varepsilon_{BB}= 0.5.$

What we need is to create a `Mas[i]` array to store different mass for atoms. Let's set the first 40 atoms are A-type, and the rest are B-type. 


In [ ]:
# Let's set up materials parameters, such as atomic mass, epsilon, etc.
# These parameters are set up arbitrarily. For real MD simulations,
# you will need to input realistic materials parameters.

ma = 0.5         # <-- mass
rm = 0.1         # <-- reference radius
# eps = 0.25       # <-- coefficient for energy well
r0 = rm*1.12246  # <-- equilibrium radius

epsAA = 0.25;     
epsAB = 2.5; 
# epsAB = 0.1; 
epsBB = 0.5;
mb = 1.0

# compuational box size
Lx = 1.0
Ly = 1.0


# we first import numpy library
import numpy as np
import random

# set up the number of atoms
N = 80

# create position, velocity, and acceleration arrays
Pos = np.zeros((N,2))
Vel = np.random.rand(N, 2)  # <-- random initial velocity
Vel *= 0.001 
Acc = np.random.rand(N, 2)  # <-- random initial acceleration 
Acc *= 0.0001 

Mas = np.zeros(N)
for at in range(N):
    if at < np.floor(N/2):
        Mas[at] = ma
    else:
        Mas[at] = mb
        

# set up initial atom positions. Let's make a 8x8 configuration
for i in range(10):           # <-- note that default Python loop goes from 0 to n-1
    for j in range(8):        # <-- note indent is what Python recognizes nest loop
        Pos[i*8+j,0] = j*r0*1.0 + 0.75*r0 + random.uniform(-0.01, 0.01) # <-- add some randomness
        Pos[i*8+j,1] = i*r0*0.8 + 0.75*r0 + random.uniform(-0.01, 0.01)


# save the initial atomic positions
Pos_pr = Pos.copy()
Vel_pr = Vel.copy()
Acc_pr = Acc.copy()

# Mas

**Let's visualize the initial atom configuration in the cell below.**

In [ ]:
# import matplotlib library, which is built to
# mimic Matlab's plotting functionality

import matplotlib.pyplot as plt  

# Plot results
# plt.scatter(Pos[0:40, 0], Pos[0:40, 1], c='b')
plt.plot(Pos[:40, 0], Pos[:40, 1], 'bo')
plt.plot(Pos[40:, 0], Pos[40:, 1], 'ro')
plt.xlabel("x Position")
plt.ylabel("y Position")
plt.title("Particle Positions")
plt.gca().set_aspect('equal', adjustable='box')
plt.axis([0, Lx, 0, Ly])

When you arrive this part of the code, all the necessary containers should be successfully created.

---
### Part 2: Time stepping

#### Part 2.1

The main difference between the two-phase model and single-phase model is that we will different interaction forces $$\vec{f}_{ij} = -\frac{\vec{r}_{ij}}{r_{ij}}  \frac{24 \varepsilon_{AA}}{r_m} \bigg[2 \bigg(\frac{r_m}{r_{ij}}\bigg)^{13} - \bigg(\frac{r_m}{r_{ij}}\bigg)^7 \bigg]$$ between A-A atoms.
$$\vec{f}_{ij} = -\frac{\vec{r}_{ij}}{r_{ij}}  \frac{24 \varepsilon_{BB}}{r_m} \bigg[2 \bigg(\frac{r_m}{r_{ij}}\bigg)^{13} - \bigg(\frac{r_m}{r_{ij}}\bigg)^7 \bigg]$$ between B-B atoms.
$$\vec{f}_{ij} = -\frac{\vec{r}_{ij}}{r_{ij}}  \frac{24 \varepsilon_{AB}}{r_m} \bigg[2 \bigg(\frac{r_m}{r_{ij}}\bigg)^{13} - \bigg(\frac{r_m}{r_{ij}}\bigg)^7 \bigg]$$ between A-B atoms.
So, we can simply set conditions in the previous code when calculate interatomic forces:
$$\text{if}~~at< 40 ~~\text{and}~~ne<40,~~\varepsilon = \varepsilon_{AA}$$
$$\text{if}~~at> 40 ~~\text{and}~~ne>40,~~\varepsilon = \varepsilon_{BB}$$
Other than these two cases:
$$\varepsilon = \varepsilon_{AB}$$


* Copy the single-phase code to here.
* Implement the conditions above to where the interatomic forces are calculated.
* Calculate interatomic forces between pairs. Note we only calculate those atoms
* Calculate total force as before.
* Use Newton's law to calculate acceleration of each atom: $$\vec{a}_i = \frac{\vec{f}_i}{m_i}.$$ Note that the two types of atoms have different mass.
* Update the atom positions and impose periodic boundary conditions as before.
    

**Use the code cell to make the time simulation.**

* Test the first case that
$$\varepsilon_{AA} = 0.25, \varepsilon_{AB} = 2.5, \varepsilon_{BB} = 0.5,$$
where the A-B interaction force is much larger. Do you expect A and B will mix?

* Test the second case that
$$\varepsilon_{AA} = 0.25, \varepsilon_{AB} = 0.1, \varepsilon_{BB} = 0.5,$$
where the A-B interaction force is much lower. Do you expect A and B will mix?

In [ ]:
# import libraries for plotting
from IPython.display import display, clear_output
import time

# Initialization before the loop
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)


# set up number of time steps and time step size
nstep = 2001
dt = 0.00005
r_cut = 3*r0


# get initial positions from Part 1
Pos = Pos_pr.copy() 
Vel = Vel_pr.copy()
Acc = Acc_pr.copy()
   
tm = 0
cnt = 1

# Simulation loop
for it in range(0, nstep + 1):

    # Periodic boundary conditions
    ??
            
    # Calculate inter-atom distances with periodic boundaries
    ??


    # Calculate force
    Fij.fill(0)

    for at in range(N - 1):
        for ne in range(at + 1, N):
            if 0 < dst[at,ne,2] < r_cut:
                
                if ??:
                    eps = epsAA
                elif ??:
                    esp = epsBB
                else:
                    eps = epsAB
                
                dUdr = ??
                
                Fij[at,ne,0] = ??
                Fij[at,ne,1] = ??

                Fij[ne,at,0] = -Fij[at,ne,0]
                Fij[ne,at,1] = -Fij[at,ne,1]

    # Calculate total force of each atom
    for at in range(N):

    
    # calculate acceleration of each atom
    # Acc = Fi_v/ma
    for at in range(N): 
        Acc[at,0] = Fi_v[at,0]/Mas[at]
        Acc[at,1] = Fi_v[at,1]/Mas[at]
        
    # Update positions
    for at in range(N): 
        Pos[at,0] += Vel[at,0] * dt
        Pos[at,1] += Vel[at,1] * dt

    
    # Update velocity
    for at in range(N): 
        Vel[at, 0] += Acc[at, 0] * dt
        Vel[at, 1] += Acc[at, 1] * dt

    # print(X_Y)

    # Elapsed time
    tm += dt

    # Visualization
    if it % 250 == 1:
        plt.plot(Pos[:40, 0], Pos[:40, 1], 'bo')
        plt.plot(Pos[40:, 0], Pos[40:, 1], 'ro')       
        plt.axis([0, Lx, 0, Ly])
        print(it)
   
        # Animaiton part (dosn't change)
        clear_output(wait=True) # Clear output for dynamic display
        display(fig)            # Reset display
        fig.clear()             # Prevent overlapping and layered plots
        time.sleep(0.0002)         # Sleep for half a second to slow down the animation

---

#### Part 2.2

Now, **let's use Verlet time scheme.**

* Modify the preivous Verlet method code to run two-phase simulations.

In [ ]:

# import libraries for plotting
from IPython.display import display, clear_output
import time

# Initialization before the loop
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)


# set up number of time steps and time step size
nstep = 10001
dt = 0.00005
r_cut = 3*r0


# get initial positions from Part 1
Pos = Pos_pr.copy() 
Vel = Vel_pr.copy()
Acc = Acc_pr.copy()

for at in range(N):
    Pos[at,0] = Pos_pr[at,0] + Vel[at,0]*dt + 0.5*Acc[at,0]*dt**2
    
Pos_c = Pos.copy()
Pos_p = Pos_pr.copy()

tm = 0
cnt = 1

# Simulation loop
for it in range(0, nstep + 1):

    # copy your working single-phase code here and modify it to
    # a two-phase code.
    


    

    # Elapsed time
    tm += dt

    # Visualization
    if it % 250 == 1:
        plt.plot(Pos[:40, 0], Pos[:40, 1], 'bo')
        plt.plot(Pos[40:, 0], Pos[40:, 1], 'ro')
        plt.axis([0, Lx, 0, Ly])
        print(it)

   
        # Animaiton part (dosn't change)
        clear_output(wait=True) # Clear output for dynamic display
        display(fig)            # Reset display
        fig.clear()             # Prevent overlapping and layered plots
        time.sleep(0.0002)         # Sleep for half a second to slow down the animation




Upload you final copy to the drop box on the course website. Coding Lab 1.